# AWS IoT DR - Create a private CA (PCA) in ACM

Create a private CA in [AWS Certificate Manager](https://docs.aws.amazon.com/acm/latest/userguide/acm-overview.html). You can use the private CA to issue certificates for your devices.

## Libraries

In [ ]:
from OpenSSL import crypto, SSL
from os.path import exists, join
from os import makedirs
from shutil import copy
from time import time, gmtime, localtime, strftime
import boto3
import json
import time

## Some handy functions
Functions to create keys, certificate signing requests and certificates.

In [ ]:
def create_csr(pkey, subject, digest="sha256"):
    print("subject: {}".format(subject))
    req = crypto.X509Req()
    subj = req.get_subject()
    
    for i in ['C', 'ST', 'L', 'O', 'OU', 'CN']:
        if i in subject:
            setattr(subj, i, subject[i])

    req.set_pubkey(pkey)
    req.sign(pkey, digest)
    return req


def create_certificate(req, issuerCertKey, serial, validityPeriod,
                      digest="sha256", createca=False, intermediate=False):

    issuerCert, issuerKey = issuerCertKey
    notBefore, notAfter = validityPeriod
    cert = crypto.X509()
    cert.set_version(2)
    cert.set_serial_number(serial)
    cert.gmtime_adj_notBefore(notBefore)
    cert.gmtime_adj_notAfter(notAfter)
    cert.set_issuer(issuerCert.get_subject())
    cert.set_subject(req.get_subject())
    cert.set_pubkey(req.get_pubkey())

    if createca:
        if intermediate:
            cert.add_extensions([
              crypto.X509Extension(b"basicConstraints", False, b"critical,CA:TRUE,pathlen:0"),
              crypto.X509Extension(b"subjectKeyIdentifier", False, b"hash", subject=cert),
              crypto.X509Extension(b"authorityKeyIdentifier", False, b"keyid:always,issuer", issuer=issuerCert)
            ])            
        else:
            cert.add_extensions([
              crypto.X509Extension(b"basicConstraints", False, b"CA:TRUE"),
              crypto.X509Extension(b"subjectKeyIdentifier", False, b"hash", subject=cert),
            ])
            cert.add_extensions([
              crypto.X509Extension(b"authorityKeyIdentifier", False, b"keyid:always,issuer:always", issuer=cert)
          ])
    else:
      cert.add_extensions([
          crypto.X509Extension(b"subjectKeyIdentifier", False, b"hash", subject=cert),
          crypto.X509Extension(b"authorityKeyIdentifier", False, b"keyid:always,issuer:always", issuer=issuerCert)
      ])

    cert.sign(issuerKey, digest)
    return cert


def load_ca(cert_dir, cert_file, key_file):
    cacert = crypto.load_certificate(crypto.FILETYPE_PEM,
        open(join(cert_dir, cert_file), "r").read())
    cakey = crypto.load_privatekey(crypto.FILETYPE_PEM,
        open(join(cert_dir, key_file), "r").read())
    return cacert, cakey


def create_ca(cert_dir, cert_file, key_file, subject, duration):

    if exists(join(cert_dir, cert_file)) and exists(join(cert_dir, key_file)):
        print("CA {} exists".format(subject['CN']))
        return load_ca(cert_dir, cert_file, key_file)

    if not exists(cert_dir):
        print("creating directory: {}".format(cert_dir))
        makedirs(cert_dir)

    print("Create CA {}".format(subject['CN']))
    cakey = crypto.PKey()
    cakey.generate_key(crypto.TYPE_RSA, 2048)
    careq = create_csr(cakey, subject)


    serial = int(time.strftime("%y%m%d%H%M%S", localtime()))
    cacert = create_certificate(careq, (careq, cakey), serial, (0, 60*60*24*365*duration), createca=True)
    open(join(cert_dir, cert_file), "wt").write(
        crypto.dump_certificate(crypto.FILETYPE_PEM, cacert).decode('utf-8'))
    open(join(cert_dir, key_file), "wt").write(
        crypto.dump_privatekey(crypto.FILETYPE_PEM, cakey).decode('utf-8'))
    print('created CA with private key "'+key_file+'" and certificate "'+cert_file+'"')

    return cacert, cakey


def create_priv_key_and_csr(cert_dir, csr_file, key_file, subject):
    if not exists(cert_dir):
        print("creating directory: {}".format(cert_dir))
        makedirs(cert_dir)
        
    priv_key = crypto.PKey()
    priv_key.generate_key(crypto.TYPE_RSA, 2048)
    #print(crypto.dump_privatekey(crypto.FILETYPE_PEM, priv_key))

    key_file = join(cert_dir, key_file)
    f = open(key_file,"w")
    f.write(crypto.dump_privatekey(crypto.FILETYPE_PEM, priv_key))
    f.close()
    
    csr = create_csr(priv_key, subject)

    csr_file = join(cert_dir, csr_file)
    f= open(csr_file,"w")
    f.write(crypto.dump_certificate_request(crypto.FILETYPE_PEM, csr))
    f.close()
    
    return crypto.dump_certificate_request(crypto.FILETYPE_PEM, csr)

## Shared variables

Import shared variables into this notebook.

In [ ]:
%store -r config

print("config: {}".format(json.dumps(config, indent=4, default=str)))

## Create top level CA

ACM PCA supports subordinated CAs. Therefore we need to create a CA outside of ACM which is being used to chain the subordinated CA.

Create a CA locally which will be used to chain with the subordinate CA from ACM. The PCA will be created later. CA key and certificate will be stored in the directory that is defined by the variable *CA_directory*. A duration of 20 years is used for the top level CA.

In [ ]:
ca_cert_crypto, ca_key_crypto = create_ca(
    config['CA_directory'], 
    config['CA_cert'], 
    config['CA_key'], 
    config['CA_subject'], 20)

## Boto3 client
Create a boto3 client for the acm-pca service endpoint.

In [ ]:
c_acm_pca = boto3.client('acm-pca', region_name = config['aws_region_pca'])

## Subordinate CA
Create a private subordinated CA in ACM. You can change the `Subject` if required.

In [ ]:
response = c_acm_pca.create_certificate_authority(
    CertificateAuthorityConfiguration={
        'KeyAlgorithm': 'RSA_2048',
        'SigningAlgorithm': 'SHA256WITHRSA',
        'Subject': {
            'Country': 'DE',
            'Organization': 'AWS',
            'OrganizationalUnit': 'IoT',
            'State': 'Berlin',
            'CommonName': 'Device CA for IoT DR',
            'Locality': 'Berlin'
        }
    },
    RevocationConfiguration={
        'CrlConfiguration': {
            'Enabled': False
        }
    },
    CertificateAuthorityType='SUBORDINATE',
    IdempotencyToken='MySubOrdinateCA'
)

print("response: {}".format(json.dumps(response, indent=4, default=str)))

pca_arn = response['CertificateAuthorityArn']
print("pca_arn: {}".format(pca_arn))

## Retrieve CSR for PCA
After the PCA has been created you need to get the CSR for your private CA. This CSR must be signed by the top level CA that has been created in the first step. After creating a cert from the CSR, the certificate must be imported into ACM to activate the private CA. 
CSR is written to file.

In [ ]:
response = c_acm_pca.get_certificate_authority_csr(
    CertificateAuthorityArn = pca_arn
)
print("response: {}".format(json.dumps(response, indent=4, default=str)))
subca_csr = response['Csr']
print("subca_csr: {}".format(subca_csr))

file_subca_csr = join(config['CA_directory'], 'subca_csr.pem')
f = open(file_subca_csr,"w")
f.write(subca_csr)
f.close()

## Create certificate for subordinated CA
Based on the CSR from the subordinate CA we need to create a certificate signed by the top level CA. 

### Note: Subject of the CSR and CRT must be the same. Otherwise ACM PCA will not accept the cert.

Duration of the certificate for the PCA cert will be set to 10 years.

In [ ]:
subca_req = crypto.load_certificate_request(crypto.FILETYPE_PEM, subca_csr)
serial = int(time.strftime("%y%m%d%H%M%S", localtime()))
subca_cert_crypto = create_certificate(subca_req, (ca_cert_crypto, ca_key_crypto), serial, (0, 60*60*24*365*10), createca=True, intermediate=True)

subca_cert = crypto.dump_certificate(crypto.FILETYPE_PEM, subca_cert_crypto).decode()
print(subca_cert)

file_subca_crt = join(config['CA_directory'], 'subca_crt.pem')
f = open(file_subca_crt,"w")
f.write(subca_cert)
f.close()

## Import CA into ACM
To import the certificate that has been created from the CSR from the subordinated CA into ACM the following parameters are required:

* ARN of the private CA in ACM
* Certificate based on the CSR from the private CA
* Certificate chain of all certificates used to sign the CSR from the private CA. In this case the certificate from the top level CA is the one that is needed

### Last check to verify all parameters are set

In [ ]:
print("pca_arn: {}".format(pca_arn))
print(subca_cert)
ca_cert = crypto.dump_certificate(crypto.FILETYPE_PEM, ca_cert_crypto).decode()
print(ca_cert)

### Import the certificate

In [ ]:
response = c_acm_pca.import_certificate_authority_certificate(
    CertificateAuthorityArn = pca_arn,
    Certificate = subca_cert,
    CertificateChain = ca_cert
)

print("response: {}".format(json.dumps(response, indent=4, default=str)))

### Verify 
Verify that the certificate has been imported correctly by describing the private CA in ACM.

In [ ]:
response = c_acm_pca.get_certificate_authority_certificate(
    CertificateAuthorityArn = pca_arn
)
print("response: {}\n".format(json.dumps(response, indent=4, default=str)))
pca_certificate = response['Certificate']
print("pca_certificate:\n{}".format(pca_certificate))